In [11]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from joblib import dump, load
from PIL import Image, ImageFilter, ImageChops
from skimage import feature
from random import random
from random import uniform
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

### Read images

In [9]:
def read_images(input_path):
    """
    Read images in the input_path,
    save image, patient of each image and the class (group/labels)

    Params:
    input_path = path to the original images

    Return:
    images = list of all images
    labels = list with class for each image
    """

    # Lists to save images, patients and labels
    images = []
    labels = []

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory
        if os.path.isdir(class_path):

            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

                # Append image, patient id and class to list
                images.append(image)
                labels.append(class_dir)

    return (images, labels)

In [4]:
%%bash
#wget https://www.inf.ufpr.br/vsa20/dataset.tar.gz
tar -xf /content/dataset.tar.gz

### Preprocessing

### Extract features

In [5]:
# Extract grayscale features
def extract_grayscale(data):

    ft_gray = []

    for image in data:

        shape = image.shape
        ft_gray.append(np.reshape(image, (shape[0]*shape[1])))

    return ft_gray

In [6]:
# Extract feature: length of the edges
def extract_edges_len(data):

    ft_edges_len = []

    for image in data:

        edges = feature.canny(image)

        # Extract the edge pixels
        edge_pixels = edges.nonzero()

        # Calculate the edge length for each edge pixel
        edge_lengths = []

        for i, j in zip(*edge_pixels):
            # Get the coordinates of the current pixel
            x, y = j, i

            # Check if there's a next pixel in the edge segment
            if x > 0:
                # Get the coordinates of the next pixel
                x2 = edge_pixels[0][x - 1]
                y2 = edge_pixels[1][x - 1]
            else:
                # Handle the edge case where there's no next pixel
                x2 = edge_pixels[0][-1]
                y2 = edge_pixels[1][-1]

            # Calculate the distance between the two pixels
            distance = np.sqrt((x2 - x)**2 + (y2 - y)**2)

            # Add the distance to the list of edge lengths
            edge_lengths.append(distance)

        ft_edges_len.append(edge_lengths)

    return ft_edges_len

In [ ]:
# Extract features: curvature of the edges
from scipy.interpolate import CubicBezier
def extract_edges_curv(data):
    edges = feature.canny(image)


In [27]:
# Extract features: LBP

def extract_lbp(data, eps=1e-7, points=24, radius=8):

    ft_lbp = []

    for image in data:

        lbp = feature.local_binary_pattern(image,
                                           points,
                                           radius,
                                           method="uniform")

        (hist, _) = np.histogram(lbp.ravel(),
                                 bins = np.arange(0, points + 3),
                                                 range=(0, points + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        ft_lbp.append(hist)

    # return the histogram of Local Binary Patterns
    return ft_lbp

In [12]:
x_train, y_train = read_images("/content/dataset/train")

In [13]:
ft_gray = extract_grayscale(x_train)

In [18]:
ft_edges_len = extract_edges_len(x_train)

In [37]:
ft_lbp = extract_lbp(x_train)

Get feature names and table

In [38]:
max_edges_count = 0
for sample in ft_edges_len:
    edges_count = len(sample)

    if edges_count > max_edges_count:
        max_edges_count = edges_count

names = []

names_gray = [f'gray{i}' for i in range(len(ft_gray[0]))]
names_edges = [f'edges{i}' for i in range(max_edges_count)]
names_lbp = [f'lbp{i}' for i in range(len(ft_lbp[0]))]

names.append(names_gray)
names.append(names_edges)
names.append(names_lbp)

In [ ]:
df_gray = pd.DataFrame(ft_gray, columns = names_gray)
df_edges = pd.DataFrame(ft_edges_len, columns = names_edges)
df_lbp = pd.DataFrame(ft_lbp, columns = names_lbp)

df = pd.concat([df_gray, df_edges, df_lbp], axis = 1)

# Create a LabelEncoder object
df['label'] = y_train
le = LabelEncoder()
le.fit(df['label'])
df['label'] = le.transform(df['label'])

df = df.reset_index()

In [ ]:
df.to_csv("/content/df.csv")

### PSO

In [ ]:
# numero total de features disponiveis
max_feature_id = 50

In [ ]:
# number of dimensions
# i.e. feature number for each particle
n_dimensions = 3

# initial particles position
# since we can't use the same feature repeated,
# the initial position has features ranging from 0 to n_features
initial_pos = []
for i in range(n_dimensions):
    initial_pos.append(i)

# min and max values for the features
# 0 is the id for the first feature,
# and max_feature_id is the id for the last feature
bounds = [(0, max_feature_id)]*n_dimensions



In [ ]:
def cost_function(x):
    total=0
    for i in range(len(x)):
        total+=x[i]**2
    return total

In [ ]:
class Particle:
    def __init__(self, initial_pos):

        self.position_i = []          # particle position, i.e. features
        self.velocity_i = []          # particle velocity
        self.pos_best_i = []          # best position individual
        self.err_best_i = -1          # best error individual
        self.err_i = -1               # error individual

        # intiialize position
        self.position_i = initial_pos
        # intiialize velocity as values between -1 and 1
        for i in range(0, n_dimensions):
            self.velocity_i.append(uniform(-1,1))

    # evaluate current fitness
    def evaluate(self, costFunc):
        self.err_i = costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i == -1:
            self.pos_best_i = self.position_i.copy()
            self.err_best_i = self.err_i

    # update new particle velocity
    def update_velocity(self, pos_best_g, w, c1, c2):

        # constant inertia weight (how much to weigh the previous velocity)
        # cognitive constant (influences pbest)
        # social constant (influences gbest)

        for i in range(0, n_dimensions):

            # non-deterministic values to prevent particles
            # from getting stuck in local optima
            r1 = random()
            r2 = random()

            # update cognitive and social
            vel_cognitive = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            vel_social = c2 * r2 * (pos_best_g[i] - self.position_i[i])

            self.velocity_i[i] = w * self.velocity_i[i] + vel_cognitive + vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0, n_dimensions):
            self.position_i[i] = round(self.position_i[i] + self.velocity_i[i])

            # adjust maximum position if necessary
            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]


def minimize(cost_function, initial_pos, bounds, n_particles,
             n_dimensions, maxiter, verbose=False):

    err_best_g = -1                   # best error for group
    pos_best_g = []                   # best position for group

    # establish the swarm
    swarm = []
    for i in range(0, n_particles):
        swarm.append(Particle(initial_pos, n_dimensions))

    # begin optimization loop
    i = 0
    while i < maxiter:
        if verbose: print(f'iter: {i:>4d}, best solution: {err_best_g:10.6f}')

        # cycle through particles in swarm and evaluate fitness
        for j in range(0, n_particles):
            swarm[j].evaluate(cost_function)

            # determine if current particle is the best (globally)
            if swarm[j].err_i < err_best_g or err_best_g == -1:
                pos_best_g = swarm[j].position_i
                err_best_g = float(swarm[j].err_i)

        # cycle through swarm and update velocities and position
        for j in range(0, n_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)

        i+=1

    # print final results
    if verbose:
        print('\nFINAL SOLUTION:')
        print(f'   > {pos_best_g}')
        print(f'   > {err_best_g}\n')

    return err_best_g, pos_best_g

In [ ]:
minimize(sphere, initial, bounds, num_particles=15, maxiter=30, verbose=True)